# **Project Machine Learning Pipeline - Human Stress Prediction**

## **Import Library**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tfx.components import (CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, 
                            Transform, Trainer, Tuner, Evaluator, Pusher)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma
import os
import pandas as pd

## **Set variabel**

Mendefinisikan nama pipeline dan beberapa jalur direktori penting dalam proyek TFX (TensorFlow Extended). Variabel `PIPELINE_NAME` dan `SCHEMA_PIPELINE_NAME` masing-masing menentukan nama pipeline utama (dalam hal ini "maoelana-pipeline") dan pipeline untuk analisis skema data ("stress-tfdv-schema"). `PIPELINE_ROOT` adalah direktori tempat artefak pipeline disimpan, `METADATA_PATH` menunjuk ke lokasi file metadata yang menyimpan informasi eksekusi pipeline, dan `SERVING_MODEL_DIR` adalah direktori tujuan model yang sudah dilatih dan siap untuk disajikan (deployment).

In [2]:
PIPELINE_NAME = "maoelana-pipeline"
SCHEMA_PIPELINE_NAME = "stress-tfdv-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

Membaca file dataset bernama "Stress.csv" yang berada dalam folder "data" menggunakan library pandas dan menyimpan ke dalam variabel `stress`. Kemudian `stress.info()` digunakan untuk menampilkan ringkasan informasi dari DataFrame tersebut, termasuk jumlah baris dan kolom, nama kolom, jumlah nilai non-null di setiap kolom, tipe data tiap kolom dan yang lainnya. Hal ini berguna untuk memahami struktur dan kelengkapan data.

In [3]:
stress = pd.read_csv("data/Stress.csv")
stress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2838 non-null   object
 1   label   2838 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 44.5+ KB


In [4]:
DATA_ROOT = 'data'

Membuat `interactive_context` dari class `InteractiveContext` dalam TFX (TensorFlow Extended). Parameter `pipeline_root=PIPELINE_ROOT` menunjukkan direktori tempat artefak-artefak pipeline (seperti data hasil transformasi, model, evaluasi, dll.) akan disimpan. Hal ini sangat berguna saat membangun dan menguji pipeline secara bertahap sebelum diterapkan secara penuh.

In [5]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## **Data Ingestion**

### **ExampleGen**

Mengonfigurasi pembagian data (data split) dalam komponen ExampleGen pada pipeline TFX. `output` didefinisikan sebagai untuk `example_gen_pb2.Output`, yang menerima konfigurasi pembagian melalui parameter `split_config`. Di dalamnya, `split_config` menentukan dua bagian data: satu bernama "train" dengan 8 hash buckets (80% data), dan satu lagi bernama "eval" dengan 2 hash buckets (20% data).

In [6]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

Menjalankan komponen CsvExampleGen dalam pipeline TFX. Komponen `CsvExampleGen` digunakan untuk membaca data mentah dalam format CSV dari direktori yang ditentukan oleh `DATA_ROOT`. Parameter `output_config=output` mengatur pembagian data menjadi set pelatihan dan evaluasi sesuai konfigurasi sebelumnya (misalnya 80% train, 20% eval). Setelah komponen dibuat, `interactive_context.run(example_gen)` menjalankan komponen tersebut dalam konteks interaktif sehingga data dapat langsung diproses dan hasilnya (TFRecord) disimpan dalam direktori artefak pipeline untuk digunakan oleh komponen-komponen berikutnya seperti StatisticsGen, SchemaGen, dan Trainer.

In [7]:
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## **Data Validation**

### **StatisticGen**

Menjalankan komponen StatisticsGen dari TFX yang bertugas menghitung statistik deskriptif pada data hasil CsvExampleGen. `statistic_gen` dibuat dengan input data `examples` dari output komponen `example_gen`. Selanjutnya, `interactive_context.run(statistic_gen)` akan menjalankan komponen ini dalam mode interaktif, menghitung statistik seperti distribusi nilai, jumlah nilai null, min, max, dan lain-lain untuk setiap fitur.

In [8]:
statistic_gen = StatisticsGen(examples=example_gen.outputs["examples"])
interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

`interactive_context.show(statistic_gen.outputs['statistics'])` digunakan untuk menampilkan hasil statistik tersebut secara visual, yang berguna untuk eksplorasi awal dan deteksi anomali data.

In [9]:
interactive_context.show(statistic_gen.outputs['statistics'])

### **SchemaGen**

Menjalankan komponen SchemaGen dalam pipeline TFX, yang digunakan untuk menghasilkan skema data (data schema) berdasarkan statistik yang dihitung sebelumnya oleh komponen StatisticsGen. `schema_gen` dibuat dengan mengambil hasil statistik dari hasil komponen `statistic_gen` melalui parameter `statistics=statistic_gen.outputs["statistics"]`. Selanjutnya, `interactive_context.run(schema_gen)` menjalankan komponen ini dalam mode interaktif untuk membuat skema yang menggambarkan tipe data, batasan nilai (seperti nilai null atau nilai yang tidak valid), serta fitur yang ada dalam dataset. Skema ini berguna untuk validasi data pada langkah-langkah berikutnya dalam pipeline.

In [10]:
schema_gen = SchemaGen(statistics=statistic_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

Menampilkan hasil output dari komponen SchemaGen, yaitu skema data yang telah dibuat berdasarkan statistik yang dihitung sebelumnya. Setelah menjalankan `schema_gen`, `interactive_context.show(schema_gen.outputs["schema"])` digunakan untuk memvisualisasikan skema yang dihasilkan, yang berisi informasi tentang tipe data dan batasan setiap kolom (misalnya, apakah ada nilai yang hilang atau tidak valid).

In [11]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,,-
'text',BYTES,required,,-


### **ExampleValidator**

Menjalankan komponen ExampleValidator dalam pipeline TFX, yang bertujuan untuk memvalidasi data berdasarkan statistik dan skema yang telah dihasilkan sebelumnya. Objek `example_validator` dibuat dengan mengambil dua input: `statistics=statistic_gen.outputs["statistics"]` yang berisi statistik deskriptif data dan `schema=schema_gen.outputs["schema"]` yang berisi skema data. Komponen ini memeriksa apakah data sesuai dengan aturan yang ditentukan dalam skema, seperti validitas nilai, tipe data, dan nilai yang hilang. Setelah itu, `interactive_context.run(example_validator)` menjalankan proses validasi data, membantu mendeteksi masalah atau anomali pada data sebelum melanjutkan ke tahap selanjutnya dalam pipeline.

In [12]:
example_validator = ExampleValidator(
    statistics=statistic_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

Menampilkan hasil output dari komponen ExampleValidator, yaitu deteksi anomali yang ditemukan dalam dataset selama proses validasi. Setelah menjalankan komponen `example_validator`, `interactive_context.show(example_validator.outputs["anomalies"])` digunakan untuk menampilkan rincian anomali atau masalah yang ditemukan pada data, seperti nilai yang hilang, tipe data yang salah, atau ketidaksesuaian dengan skema yang telah ditentukan.

In [13]:
interactive_context.show(example_validator.outputs["anomalies"])

## **Data Preprocessing**

### **Transform**

Menjalankan komponen Transform dalam pipeline TFX, yang bertujuan untuk melakukan preprocessing dan transformasi data menggunakan modul yang telah dibuat. Objek `transform` dibuat dengan tiga input utama: `examples=example_gen.outputs["examples"]` yang berisi data contoh yang telah dibaca, `schema=schema_gen.outputs['schema']` yang berisi skema data untuk memastikan transformasi yang dilakukan sesuai dengan tipe data dan aturan validasi, serta `module_file=os.path.abspath(TRANSFORM_MODULE_FILE)` yang menunjuk pada file Python (`stress_transform.py`) yang berisi fungsi transformasi khusus yang akan diterapkan pada data. Selanjutnya, `interactive_context.run(transform)` menjalankan proses transformasi, yang mengubah data menjadi format yang siap untuk digunakan dalam model pelatihan.

In [14]:
TRANSFORM_MODULE_FILE = "stress_transform.py"
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\maoelana-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\b776f679a7484b40b0923c563128eabd\assets


INFO:tensorflow:Assets written to: pipelines\maoelana-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\b776f679a7484b40b0923c563128eabd\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## **Model Development**

### **Tuner**

Menjalankan komponen Tuner dalam pipeline TFX, yang bertujuan untuk melakukan hyperparameter tuning untuk model. `tuner` dibuat dengan menyediakan beberapa input, termasuk `module_file=os.path.abspath(TUNER_MODULE_FILE)` yang merujuk pada file Python (`stress_tuner.py`) yang berisi hyperparameter tuning, serta input lainnya seperti `examples=transform.outputs['transformed_examples']` (data yang sudah ditransformasi), `transform_graph=transform.outputs['transform_graph']` (grafik transformasi data), dan `schema=schema_gen.outputs['schema']` (skema data). Selain itu, `train_args=trainer_pb2.TrainArgs(splits=['train'])` dan `eval_args=trainer_pb2.EvalArgs(splits=['eval'])` menentukan pembagian data untuk pelatihan dan evaluasi. Setelah itu, `interactive_context.run(tuner)` menjalankan hyperparameter tuning berdasarkan data pelatihan dan evaluasi yang sudah dipersiapkan.

In [15]:
TUNER_MODULE_FILE = "stress_tuner.py"
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
)
interactive_context.run(tuner)

Trial 20 Complete [00h 00m 03s]
val_binary_accuracy: 0.7704347968101501

Best val_binary_accuracy So Far: 0.7773913145065308
Total elapsed time: 00h 01m 10s
Results summary
Results in pipelines\maoelana-pipeline\.temp\6\stress_sentiment_random_search
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
unit_1: 128
unit_2: 32
unit_3: 32
learning_rate: 0.0005
Score: 0.7773913145065308

Trial 18 summary
Hyperparameters:
unit_1: 256
unit_2: 32
unit_3: 16
learning_rate: 0.0001
Score: 0.7773913145065308

Trial 03 summary
Hyperparameters:
unit_1: 128
unit_2: 32
unit_3: 64
learning_rate: 0.001
Score: 0.7756521701812744

Trial 08 summary
Hyperparameters:
unit_1: 128
unit_2: 32
unit_3: 32
learning_rate: 0.001
Score: 0.7756521701812744

Trial 09 summary
Hyperparameters:
unit_1: 128
unit_2: 128
unit_3: 16
learning_rate: 0.0005
Score: 0.7756521701812744

Trial 17 summary
Hyperparameters:
unit_1: 256
unit_2: 128
unit_3: 64
learning_rate: 0.

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### **Trainer**

Menjalankan komponen Trainer dalam pipeline TFX, yang bertujuan untuk melatih model menggunakan data yang telah ditransformasi dan menggunakan parameter terbaik yang ditemukan oleh komponen Tuner. `trainer` dibuat dengan menyediakan beberapa input penting, yaitu `module_file=os.path.abspath(TRAINER_MODULE_FILE)` yang menunjuk pada file Python (`stress_trainer.py`) yang berisi pelatihan model, serta input lainnya seperti `examples=transform.outputs['transformed_examples']` (data yang sudah ditransformasi), `transform_graph=transform.outputs['transform_graph']` (grafik transformasi data), dan `schema=schema_gen.outputs['schema']` (skema data). Selain itu, `hyperparameters=tuner.outputs['best_hyperparameters']` menyediakan parameter terbaik dari pencarian yang dilakukan oleh komponen Tuner. Pembagian data untuk pelatihan dan evaluasi ditentukan melalui `train_args=trainer_pb2.TrainArgs(splits=['train'])` dan `eval_args=trainer_pb2.EvalArgs(splits=['eval'])`. Setelah itu, `interactive_context.run(trainer)` menjalankan proses pelatihan model menggunakan konfigurasi dan data yang telah dipersiapkan.

In [16]:
TRAINER_MODULE_FILE = "stress_trainer.py"
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 128)               2176


Epoch 1: val_binary_accuracy improved from -inf to 0.76174, saving model to pipelines\maoelana-pipeline\Trainer\model\7\Format-Serving
INFO:tensorflow:Assets written to: pipelines\maoelana-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\maoelana-pipeline\Trainer\model\7\Format-Serving\assets


1000/1000 [==============================] - 2s 2ms/step - loss: 0.4652 - binary_accuracy: 0.7701 - val_loss: 0.5405 - val_binary_accuracy: 0.7617 - lr: 5.0000e-04
INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\maoelana-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\maoelana-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

**Note :** "Your input ran out of data; interrupting training" yang muncul selama proses pelatihan terjadi karena jumlah batch yang tidak mencukupi untuk melanjutkan pelatihan. Jika ingin model terus dilatih meskipun data batch habis, bisa gunakan menggunakan metode `.repeat()` pada dataset di dalam fungsi `input_fn`. Tetapi dengan catatan bahwa penggunaan `.repeat()` akan membuat proses pelatihan menjadi lebih berat karena dataset akan diproses berulang kali tanpa henti hingga pelatihan selesai dan sebagai saran gunakan GPU jika ingin `.repeat()` dataset.

### **Resolver**

Menjalankan komponen Resolver dalam pipeline TFX, yang digunakan untuk memilih model terbaru yang telah diberkati (blessed) untuk digunakan dalam tahap evaluasi atau prediksi. `model_resolver` dibuat dengan menggunakan strategi `LatestBlessedModelStrategy`, yang memilih model terbaru yang telah diberkati (BLESSED) dari `model` dan `model_blessing`. `Channel(type=Model)` dan `Channel(type=ModelBlessing)` masing-masing merujuk pada channel yang berisi model dan informasi berkaitan dengan status model blessed. Fungsi `with_id('Latest_blessed_model_resolver')` memberi ID pada resolver untuk mempermudah identifikasi. Setelah objek dibuat, `interactive_context.run(model_resolver)` menjalankan komponen ini untuk menyelesaikan resolver dan memilih model yang tepat berdasarkan strategi pemilihan yang telah ditentukan.

In [17]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### **Evaluator**

Mengonfigurasi evaluasi model menggunakan `tfma.EvalConfig` dari TensorFlow Model Analysis (TFMA), yang digunakan untuk mengevaluasi kinerja model berdasarkan metrik tertentu. `model_specs` mengonfigurasi spesifikasi model, di mana `label_key='label'` merujuk pada kolom target (label) dalam data evaluasi. `slicing_specs=[tfma.SlicingSpec()]` memungkinkan evaluasi model berdasarkan bagian data tertentu, seperti kategori fitur. Selanjutnya, `metrics_specs` menentukan metrik yang akan dihitung selama evaluasi, termasuk `ExampleCount`, `AUC`, `FalsePositives`, `TruePositives`, `FalseNegatives`, dan `TrueNegatives`. Metrik tambahan `BinaryAccuracy` juga diatur dengan ambang batas (threshold) untuk menilai akurasi model, dengan kondisi spesifik untuk perubahan metrik (Agar model diberkati **(BLESSED)**). `value_threshold` mengatur ambang batas nilai untuk menentukan apakah akurasi diterima, dan `change_threshold` mengatur perubahan yang diperlukan untuk menganggap metrik meningkat.

In [18]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy', 
                            threshold=tfma.MetricThreshold(
                                value_threshold=tfma.GenericValueThreshold(
                                    lower_bound={'value': 0.5}
                                ),
                                change_threshold=tfma.GenericChangeThreshold(
                                    direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                    absolute={'value': 0.0001}
                                )
                            )
            )
        ])
    ]
)

Menjalankan komponen Evaluator dalam pipeline TFX, yang digunakan untuk mengevaluasi kinerja model berdasarkan konfigurasi evaluasi yang telah disiapkan. `evaluator` dibuat dengan menyediakan beberapa input utama, yaitu `examples=example_gen.outputs['examples']` yang berisi data contoh untuk evaluasi, `model=trainer.outputs['model']` yang merujuk pada model yang telah dilatih, dan `baseline_model=model_resolver.outputs['model']` yang berisi model dasar (baseline) untuk perbandingan. Selain itu, `eval_config=eval_config` menentukan konfigurasi evaluasi, termasuk metrik yang akan digunakan dan cara data pisahkan. Setelah itu, `interactive_context.run(evaluator)` menjalankan komponen ini untuk mengevaluasi performa model berdasarkan konfigurasi yang telah disusun, serta membandingkan kinerjanya dengan model baseline.

In [19]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

**Note:** Berdasarkan hasil evaluator, model akhir yang diperoleh bisa saja tidak diberkati **(NOT_BLESSED)**. Ini biasanya terjadi ketika model yang dievaluasi tidak memenuhi kriteria tertentu yang ditetapkan selama proses evaluasi, seperti metrik akurasi atau kesesuaian yang tidak mencapai (threshold) ambang batas yang telah ditentukan sebelumnya. Dalam hal ini, meskipun model berhasil dilatih dan dievaluasi, model tersebut tidak dianggap cukup baik untuk digunakan dalam produksi atau untuk proses selanjutnya, seperti pendorongan (pushing) ke server produksi.

### **TFMA Visualization**

Kode ini digunakan untuk menampilkan hasil evaluasi model secara lebih mendalam lagi setelah menjalankan komponen Evaluator. Pertama, `eval_result = evaluator.outputs['evaluation'].get()[0].uri` mengambil hasil evaluasi dari komponen Evaluator dan mendapatkan URI dari evaluasi pertama. Selanjutnya, `tfma_result = tfma.load_eval_result(eval_result)` memuat hasil evaluasi yang disimpan pada URI tersebut menggunakan TensorFlow Model Analysis (TFMA). Setelah hasil evaluasi disimpan, `tfma.view.render_slicing_metrics(tfma_result)` digunakan untuk menampilkan metrik evaluasi model berdasarkan pemotongan (slicing), memberikan insight tentang performa model pada subset data. Terakhir, `tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)` digunakan untuk memvisualisasikan indikator keadilan (fairness) dari model, memberikan gambaran tentang apakah model berperilaku secara adil terhadap kelas dalam data.

In [20]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

## **Model Deployment**

### **Pusher**

Kode ini menjalankan komponen Pusher dalam pipeline TFX, yang digunakan untuk menyimpan model yang telah dilatih dan diberkati (BLESSED) ke lokasi penyimpanan tertentu, seperti file sistem. `pusher` dibuat dengan beberapa input penting: `model=trainer.outputs['model']` yang merujuk pada model yang telah dilatih, `model_blessing=evaluator.outputs['blessing']` yang memastikan bahwa model telah berhasil diuji dan diberkati (BLESSED), serta `push_destination=pusher_pb2.PushDestination(filesystem=pusher_pb2.PushDestination.Filesystem(base_directory="serving_model/stress-prediction-model"))` yang menentukan tujuan penyimpanan model, dalam hal ini ke direktori lokal `serving_model/stress-prediction-model`. Setelah `pusher` dibuat, `interactive_context.run(pusher)` menjalankan proses pendorongan (pushing) model ke lokasi yang telah ditentukan agar siap untuk digunakan dalam lingkungan produksi untuk prediksi atau testing.

In [21]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory="serving_model/stress-prediction-model"
        )
    )
)
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))